# 打印机自动化安装研究

（图床链接已失效，关键的东西还是放在GitHub上靠谱...）

## 前言

初次安装打印机，起先我们需要数分钟来浏览阅读说明，然后再花费[片刻时间不等](https://www.bing.com/search?q=%E7%89%87%E5%88%BB%E7%9A%84%E5%85%B7%E4%BD%93%E6%97%B6%E9%97%B4)进行初次的实机操作。但打印机属于精密仪器，而且有别于通常的电视、冰箱这类单一功能家电，它对环境配置、设置参数灵敏度高，操作不当极易造成设备异常与故障。除带来的维修成本外，且由于打印机与多项纸质文书与业务挂钩，这就较大可能会导致企业生产计划延误、产品质量下降和客户投诉等问题，因此企业信息部通常会由专人专职负责保养与管理，以最大程度地减少损坏发生的概率，并及时进行维修和更换。

主流厂家的打印机在质量方面，同电脑主机般经久耐用。在实际工作中，遇到的与打印机关联的问题，占比最高的两类：驱动响应阻塞（重启、重装解决）、主机建立打印机通信的网络配置。

图表统计数据为2021年1月至10月截止，小微企业120人不等。

![](https://www.hualigs.cn/image/6488124bb8bdf.jpg)

一般而言，重启打印机、重装打印机驱动，并不需要非常专门的技术。对于普通用户而言，了解一些基础的网络、软件知识就可以完成打印机配置。从上述现象也造成了另一个习焉不察的问题就是，多数公司员工匮乏电脑与打印机之间的基础知识与应用经验，基础故障时无法有效地解决问题；绝大数情况只能等待专职人员处理，从而浪费时间和资源；在知识储备不足的情况下，自行盲目操作又有可能使问题放大的风险。


为此，我能想到的方案有三：一、聘请专员（包括但不限于自聘、外包），二、员工指导培训，三、针对主要问题，进行自动化处理。

* 方案一，专员响应最快，人力成本高，不适用于小微企业。
* 方案二，培训占用集体及个人时间较长，点到为止又会流于形式 

方案三虽说需要占用软件开发与测试、运维的额外时间专门处理，承担了一定系统、打印机厂商的技术负债，但有效缩短了专员赶赴现场、基础故障排查，并最终解决问题的时间。

考虑到打印机是企业日常办公不可或缺的精密仪器设备，对互联主机设备环境敏感，频发基础环境异常问题，且分析这方面自动化文章及案例较少，故选此题研究。

## 基础自动化安装网络打印机（通用驱动程序直连）

在部分情况下，即使没有安装打印机驱动程序，仍然可以连接到打印机并进行打印，这一般是操作系统本身已经包含了通用类型的打印机驱动程序。但获得最佳打印质量和功能，则还是需要安装特定的打印机具体型号驱动程序。

自动安装通用打印机驱动，如下代码所示。

In [ ]:
rundll32 printui.dll,PrintUIEntry /if  /r "IP_192.168.0.252 /m "Generic / Text Only"

## 自动化安装网络打印机驱动程序（专有驱动程序包制作）

由于惠普打印机驱动不支持静默安装，所以使用备份还原方式拟态静默安装。这类自动化需要自行打包驱动文件与执行代码。

**示例：系统：Windows10；IP：192.168.0.252；打印机：HP LaserJet M1536dnf MFP**

**首先需要手动安装一遍打印机** ，然后操作分三步：

1. 导出打印机注册表
1. 导出驱动，并定位具体型号驱动
1. 编写代码，导入步骤1注册表与步骤2驱动文件

### 1. 导出注册表

在手动完成打印机安装的情况下，执行如下指令，该指令执行后会在用户根目录生成 IP_192.168.0.252.reg 文件。

In [ ]:
reg export "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Print\Monitors\Standard TCP/IP Port\Ports\IP_192.168.0.252" "%userprofile%\IP_192.168.0.252.reg"

![ ](https://www.hualigs.cn/image/648832f889d2b.jpg)

### 2. 导出驱动，并定位具体型号驱动

导出驱动，在当前用户名的Links文件夹（由于驱动程序很多，故将其置于此目录下）

In [ ]:
dism /online /export-driver /destination:%userprofile%\Links

![ ](https://www.hualigs.cn/image/648833467e571.jpg)

从打印器服务属性查看安装打印机驱动依赖关联

![](https://www.hualigs.cn/image/648840d9ecef4.jpg)

复制配置文件名称，搜索导出的驱动是否存在此配置

![](https://www.hualigs.cn/image/648843ea9b312.jpg)

复制所有与配置文件相关的驱动文件到自定义目录，例如：C:\Users\nice\Documents\hpcu215u.inf

![](https://www.hualigs.cn/image/648846303b7eb.jpg)

### 3. 编写代码，导入步骤1注册表与步骤2驱动文件

注意：为保持稳定性首先需重置spooler服务 `net stop spooler & net start spooler`

编写代码

```
@REM 静默模式导入步骤一的注册表
regedit /s"%userprofile%\IP_192.168.0.252.reg

@REM 将步骤二的自定义目录的有关打印机配置驱动进行导入式安装
rundll32 printui.dll,PrintUIEntry /if /f %userprofile%\Documents\hpcu215u.inf\hpcu215u.inf /r "IP_192.168.0.252" /m "HP Universal Printing PCL 6" 
```

安装成功如下图所示

![](https://www.hualigs.cn/image/64884c7e791be.jpg)

### 设置默认打印机

```
echo 设置成默认打印机
reg add "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Windows" /v Device /t REG_SZ /d "HP Universal Printing PCL 6,winspool,Ne09:" /f
```

![](https://www.hualigs.cn/image/64884e0932d2a.jpg)


**注意：win11及以上系统，需要关闭让Windows管理我的打印机，设置默认打印机的注册表才能生效**

In [ ]:
echo 关闭让Windows管理我的打印机
reg add "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Windows" /v LegacyDefaultPrinterMode/t REG_DWORD /d 1 /f

echo 设置成默认打印机
reg add "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Windows" /v Device /t REG_SZ /d "HP Universal Printing PCL 6,winspool,Ne09:" /f


至此，后续可将代码与注册表分别放入步骤二所导出的打印机驱动文件夹内，打包成压缩包，以供一键自动化安装。

## 共享打印机

**示例：**

* **系统：Windows10；连接主机IP：192.168.0.252；打印机：HP Laserjet P1606dn**
* **共享账号：用户名：ABC；密码：123456**


### 1. 获取到共享打印机名称

如果仅用bat来动态获取共享打印机名称将会大大增加逻辑复杂度，打印机驱动程序一般情况少有变动。首先建立持久化连接，然后通过`net view` 查看共享主机资源。通过类别来知道具体的共享打印机名称。

In [ ]:
net use \\192.168.0.158 /user:abc 123456 /persistent:yes
net view \\192.168.0.158

![](https://www.hualigs.cn/image/648967d44c149.jpg)

拿到打印机名称


### 3. 安装

In [ ]:
rundll32 printui.dll,PrintUIEntry /y /in /n "\\192.168.0.158\HP LaserJet Professional P1606dn (副本 1)" /q

![](https://www.hualigs.cn/image/64897e6eaed2f.jpg)

## 附录

### 其他调试运行的报错处理

清理打印池，不清理打印池会影响驱动安装。

In [ ]:
del /s /q C:\Windows\System32\spool\printers\*

删除spool中的printers目录里的所有文件

![](https://www.hualigs.cn/image/64916ec752d1c.jpg)

如下报错，针对特定品牌机存在“储存区不可用”的问题，需要解除强制驱动签名设置，即F8高级启动项，禁用驱动签名设置。详情参考：[csdn -win10安装打印机驱动程序失败“试图将读懂程序添加到存储区时遇到问题”](https://blog.csdn.net/weixin_43215344/article/details/119637939)

![](https://www.hualigs.cn/image/649170f82c5f8.jpg)

解除后，允许安装即可。

![](https://www.hualigs.cn/image/6491716d67dea.jpg)



### 设置不带密码的共享打印机简要说明

就有关“设置不带密码的共享打印机”，常规情况下需要进入本地组策略进行设置，解除特定用户组及用户的限制，对网络访问设置（组策略：“本地账户的共享和安全模型”）特殊的身份验证，并放行“文件和打印机共享”的防火墙有关条目，方可设置成功。另一方面，由于自动化设置共享打印机涉及多方面的综合知识整理分析与集成应用，需要对硬件厂商驱动支持、组策略、编程语言有着全方位体系化了解与应用。

### 参考资料

* [百度经验 - 如何不用输入主机账号密码连接共享打印机](https://jingyan.baidu.com/article/fec4bce26ace28f2608d8b71.html)
* [百度经验 - 设置打印机共享,适用Win7、Vista、xp，不用密码](https://jingyan.baidu.com/article/b907e627d4cdd446e7891ce7.html)
* [Elmagnifico's Blog - 注册表追踪](https://elmagnifico.tech/2022/05/15/Registry-Track/)
* [HP - 管理和配置打印机驱动程序](http://h10032.www1.hp.com/ctg/Manual/c01605519.pdf)
* [4sysops - Install, remove, list, and set default printer with PowerShell](https://4sysops.com/archives/install-remove-list-and-set-default-printer-with-powershell/)
* [wavecn - Windows 系统安全基线及软件工具介绍](http://www.wavecn.com/content.php?id=268)
* [serverfault.com - How can I locate Registry key for Group policy settings?](https://serverfault.com/questions/911131/how-can-i-locate-registry-key-for-group-policy-settings)
* [serverfault.com - How can I view the mapping between GPOs and registry settings?](https://serverfault.com/questions/937991/how-can-i-view-the-mapping-between-gpos-and-registry-settings)
* [网上校管系统资料库（香港）- IT Security - Windows Server Hardening Guide (English Version)](https://cdr.websams.edb.gov.hk/Files/Sys_sec/Sys_operation/Windows%20Server%20Hardening%20Guide.pdf)
* [How to import group policy module in powershell](https://tonygonzalez0379.com/2018/09/04/how-to-import-group-policy-module-in-powershell/)

In [ ]:
# 关闭防火墙
# netsh advfirewall firewall set rule group="File and Printer Sharing" new enable=Yes profile=domain
netsh advfirewall firewall set rule group="核心网络诊断" new enable=Yes
netsh advfirewall firewall set rule group="文件和打印机共享" new enable=Yes

# 开启guest账户
net user guest /active:yes

# 设置共享打印机
Set-Printer -Name "HP LaserJet M1536dnf MFP (FBC07F)" -Shared $True -ShareName "HP LaserJet M1536dnf MFP (FBC07F)"

In [ ]:
@REM ping 5 次以上，如果不在线，就退出
ping -n 5 %printerIP% > nul
if %errorlevel% neq 0 (
    echo 不在线 %printerIP% 稍后试
    pause
    exit
)

组策略指令支持需另行配置额外的模块，并且需要一个完整初始的Windows环境（最好是Windows server）才能确保顺利进行安装成功。可参考如下方案：

* [prajwaldesai - install-group-policy-management-console-on-windows-11](https://www.prajwaldesai.com/install-group-policy-management-console-on-windows-11/)
* [系统极客 - 如何在 Windows 10 Version 1809 中安装 RSAT 远程服务器管理工具](https://www.sysgeek.cn/windows-10-1809-install-rsat/)
* [learn.microsoft - 远程服务器管理工具](https://learn.microsoft.com/zh-cn/windows-server/remote/remote-server-administration-tools#install-uninstall-and-turn-offon-rsat-tools)